In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows as df_to_row

In [78]:
from szp_funcs import path_to_data, to_double, print_df

In [3]:
res = pd.read_excel(path_to_data + 'mar.xlsx')

In [6]:
res['stv'] = res.apply(lambda row: to_double(row, 'stv'), axis=1)

In [8]:
jobs = ['Старший педагог дополнительного образования', 'Педагог дополнительного образования']

In [7]:
res

,inn,snils,sum,job,day,type,stv,status,status_pref
0,7734408766,001-879-913 57,111377.54,Преподаватель,0,Основное место работы,1.0,Работа,Работа
1,7734408766,001-896-420 39,130096.11,Преподаватель,0,Основное место работы,1.0,Работа,Работа
2,7734408766,002-050-383 67,68033.97,Преподаватель,1,Основное место работы,1.0,Работа,Работа
3,7734408766,002-065-271 85,92051.69,Преподаватель,1,Основное место работы,1.0,Работа,Работа
4,7734408766,002-845-525 25,142496.37,Преподаватель,1,Основное место работы,1.0,Работа,Работа
...,...,...,...,...,...,...,...,...,...
199586,7722069597,186-373-917 14,91600.00,Специалист по интернет-маркетингу,1,Основное место работы,1.0,Работа,Работа
199587,7722069597,187-703-552 01,78220.67,Воспитатель,0,Основное место работы,1.0,Отпуск учебный оплачиваемый,Работа
199588,7722069597,198-391-256 24,83680.00,Воспитатель,1,Основное место работы,1.0,Работа,Работа
199589,7722069597,202-693-820 52,89965.14,Учитель,1,Основное место работы,1.0,Работа,Работа


In [59]:
stop = ['Увольнение']

In [69]:
res[(res.job.isin(jobs)) & (res.status.isna() == False) & (res.status.isin(stop) == False)][['inn', 'type', 'snils']].nunique()

inn        550
type         3
snils    18255
dtype: int64

In [74]:
ans = pd.DataFrame()
for job in jobs:
    t_snils = res[(res.job == job) & (res.status.isna() == False) & (res.status.isin(stop) == False)][['inn', 'type', 'snils']]
    t_stv = res[(res.job == job) & (res.status.isna() == False) & (res.status.isin(stop) == False)][['inn', 'type', 'stv']]
    t_stv_sum = t_stv.groupby(['inn', 'type']).sum().unstack().fillna(0.0)
    t_stv_sum.columns = ['_'.join(s) for s in t_stv_sum.columns]
    t_snils_unq = t_snils.groupby(['inn', 'type']).nunique().unstack().fillna(0).astype(int)
    t_snils_unq.columns = ['_'.join(s) for s in t_snils_unq.columns]
    t_stv_sum_all = t_stv[['inn', 'stv']].groupby('inn').sum()
    t_snils_unq_all = t_snils[['inn', 'snils']].groupby('inn').nunique()
    t_snils_unq.columns = [job + '_' + s for s in t_snils_unq.columns]
    t_snils_unq_all = t_snils_unq_all.rename(columns={'snils': job + '_snils'})
    t_stv_sum.columns = [job + '_' + s for s in t_stv_sum.columns]
    t_stv_sum_all = t_stv_sum_all.rename(columns={'stv': job + '_stv'})
    if len(ans) == 0:
        ans = pd.merge(t_stv_sum, t_stv_sum_all, how='right', on='inn')
        ans = pd.merge(ans, t_snils_unq, how='right', on='inn')
        ans = pd.merge(ans, t_snils_unq_all, how='right', on='inn')
        continue
    ans = pd.merge(ans, t_stv_sum, how='right', on='inn')
    ans = pd.merge(ans, t_stv_sum_all, how='right', on='inn')
    ans = pd.merge(ans, t_snils_unq, how='right', on='inn')
    ans = pd.merge(ans, t_snils_unq_all, how='right', on='inn')

In [76]:
ans = ans.fillna(0.0)

In [77]:
ans

,Старший педагог дополнительного образования_stv_Внешнее совместительство,Старший педагог дополнительного образования_stv_Внутреннее совместительство,Старший педагог дополнительного образования_stv_Основное место работы,Старший педагог дополнительного образования_stv,Старший педагог дополнительного образования_snils_Внешнее совместительство,Старший педагог дополнительного образования_snils_Внутреннее совместительство,Старший педагог дополнительного образования_snils_Основное место работы,Старший педагог дополнительного образования_snils,Педагог дополнительного образования_stv_Внешнее совместительство,Педагог дополнительного образования_stv_Внутреннее совместительство,Педагог дополнительного образования_stv_Основное место работы,Педагог дополнительного образования_stv,Педагог дополнительного образования_snils_Внешнее совместительство,Педагог дополнительного образования_snils_Внутреннее совместительство,Педагог дополнительного образования_snils_Основное место работы,Педагог дополнительного образования_snils
inn,,,,,,,,,,,,,,,,
5003021368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.833333,0.000000,3.5000,12.333333,35,0,5,40
5003021495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.310000,0.000000,15.5000,17.810000,6,0,21,27
5003021640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.2500,5.250000,0,0,6,6
5003096290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,22.975777,4.0000,27.975777,2,180,4,186
5030032182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500000,0.000000,5.5000,7.000000,3,0,6,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743935328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.875000,12.461400,17.1326,31.469000,7,77,15,99
7743937357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000,0.000000,2.0000,7.000000,10,0,2,12
7743939315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.6700,0.670000,0,0,1,1


In [79]:
print_df(ans, 'Пед доп обр по ставкам и людям')

In [9]:
stv = res[['inn', 'type', 'stv']].groupby(['inn', 'type']).sum()

In [10]:
stv

stv
inn        type                                    
5003021368 Внешнее совместительство       12.333333
           Основное место работы         970.170000
5003021495 Внешнее совместительство        8.210000
           Внутреннее совместительство    33.087500
           Основное место работы        1056.745000
...                                             ...
9715206976 Основное место работы         472.450000
9715217689 Внешнее совместительство       11.840000
           Основное место работы         250.000000
9718071371 Внешнее совместительство        0.500000
           Основное место работы         993.550000

[1561 rows x 1 columns]

In [13]:
stv_all = res[['inn', 'stv']].groupby('inn').sum()

In [61]:
stv_all.rename(columns={'stv': '1' + '_stv'})

,1_stv
inn,
5003021368,982.503333
5003021495,1098.042500
5003021640,783.050000
5003096290,580.788777
5030032182,293.000000
...,...
7743940590,310.250000
9705101759,399.000000
9715206976,573.873333


In [11]:
unique = res[['inn', 'type', 'snils']].groupby(['inn', 'type']).nunique()

In [12]:
unique

snils
inn        type                              
5003021368 Внешнее совместительство        41
           Основное место работы          971
5003021495 Внешнее совместительство        17
           Внутреннее совместительство    102
           Основное место работы         1065
...                                       ...
9715206976 Основное место работы          487
9715217689 Внешнее совместительство        29
           Основное место работы          250
9718071371 Внешнее совместительство         2
           Основное место работы          996

[1561 rows x 1 columns]

In [20]:
types = ['Внешнее совместительство', 'Внутреннее совместительство', 'Основное место работы']

In [51]:
v1 = unique.reset_index().pivot(index='inn', columns='type', values='snils')

In [52]:
v1

type,Внешнее совместительство,Внутреннее совместительство,Основное место работы,Подработка
inn,,,,
5003021368,41.0,NaN,971.0,NaN
5003021495,17.0,102.0,1065.0,NaN
5003021640,6.0,3.0,782.0,NaN
5003096290,5.0,206.0,543.0,NaN
5030032182,7.0,2.0,290.0,NaN
...,...,...,...,...
7743940590,1.0,2.0,309.0,NaN
9705101759,6.0,NaN,397.0,NaN
9715206976,30.0,261.0,487.0,NaN


In [55]:
v2.columns = ['_'.join(s) for s in v2.columns]

In [32]:
v2 = unique.unstack().fillna(0)

In [56]:
v2

,snils_Внешнее совместительство,snils_Внутреннее совместительство,snils_Основное место работы,snils_Подработка
inn,,,,
5003021368,41.0,0.0,971.0,0.0
5003021495,17.0,102.0,1065.0,0.0
5003021640,6.0,3.0,782.0,0.0
5003096290,5.0,206.0,543.0,0.0
5030032182,7.0,2.0,290.0,0.0
...,...,...,...,...
7743940590,1.0,2.0,309.0,0.0
9705101759,6.0,0.0,397.0,0.0
9715206976,30.0,261.0,487.0,0.0


In [58]:
pd.merge(v2.astype(int), unique_all, on='inn')

,snils_Внешнее совместительство,snils_Внутреннее совместительство,snils_Основное место работы,snils_Подработка,snils
inn,,,,,
5003021368,41,0,971,0,1012
5003021495,17,102,1065,0,1082
5003021640,6,3,782,0,788
5003096290,5,206,543,0,548
5030032182,7,2,290,0,297
...,...,...,...,...,...
7743940590,1,2,309,0,310
9705101759,6,0,397,0,403
9715206976,30,261,487,0,517


In [34]:
v2.columns

MultiIndex([('snils',    'Внешнее совместительство'),
            ('snils', 'Внутреннее совместительство'),
            ('snils',       'Основное место работы'),
            ('snils',                  'Подработка')],
           names=[None, 'type'])

In [28]:
pd.merge(v1, unique_all, on='inn')

,Внешнее совместительство,Внутреннее совместительство,Основное место работы,Подработка,snils
inn,,,,,
5003021368,41.0,NaN,971.0,NaN,1012
5003021495,17.0,102.0,1065.0,NaN,1082
5003021640,6.0,3.0,782.0,NaN,788
5003096290,5.0,206.0,543.0,NaN,548
5030032182,7.0,2.0,290.0,NaN,297
...,...,...,...,...,...
7743940590,1.0,2.0,309.0,NaN,310
9705101759,6.0,NaN,397.0,NaN,403
9715206976,30.0,261.0,487.0,NaN,517


In [15]:
unique_all = res[['inn', 'snils']].groupby('inn').nunique()

In [16]:
unique_all

,snils
inn,
5003021368,1012
5003021495,1082
5003021640,788
5003096290,548
5030032182,297
...,...
7743940590,310
9705101759,403
9715206976,517


In [17]:
pd.merge(stv_all, unique_all, on='inn')

,stv,snils
inn,,
5003021368,982.503333,1012
5003021495,1098.042500,1082
5003021640,783.050000,788
5003096290,580.788777,548
5030032182,293.000000,297
...,...,...
7743940590,310.250000,310
9705101759,399.000000,403
9715206976,573.873333,517
